In [1]:
from datetime import timedelta
import pandas as pd
import requests as rqs
import json
import pymongo
import datetime

myclient = pymongo.MongoClient(f'mongodb://192.168.2.1:27017',username="hterra",password="terrac0t@", unicode_decode_error_handler='ignore')
activitywatch_db = myclient["activitywatch"]
aw_watcher_window_db = myclient["activitywatch"]["aw-watcher-window"]

In [4]:
ip = 'localhost'
data = rqs.get(f'http://{ip}:5600/api/0/export').json()
host = ''
for key in data['buckets'].keys(): 
    if 'aw-watcher-window' in key:
        host = key.replace('aw-watcher-window_','')
        aw_window = data['buckets'][key]
    if 'aw-watcher-afk' in key:
        aw_afk = data['buckets'][key]
events = [{
    "timestamp": e["timestamp"],
    "duration": e["duration"],
    "host": host,
    **e["data"]
} for e in aw_window['events']]
print(str(datetime.datetime.today()),ip,"Actualizando db ...")
aw_watcher_window_db.delete_many ({"host": host})
print(str(datetime.datetime.today()),ip,"Agragando nueva data ...")
aw_watcher_window_db.insert_many(events)
print(str(datetime.datetime.today()),ip,host,"Actualizado!")


2020-02-27 14:10:34.681153 localhost Actualizando db ...
2020-02-27 14:10:35.485220 localhost Agragando nueva data ...
2020-02-27 14:10:36.489061 localhost DESKTOP-RIEHMGE Actualizado!
